In [53]:
import pandas as pd

In [54]:
data=pd.read_csv('train.csv',encoding='utf8')

In [55]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [56]:
data=data.dropna()

In [57]:
X=data.drop('label',axis=1)

In [58]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [59]:
y=data['label']
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [60]:
import tensorflow as tf


In [61]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras. models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [62]:
voc_size=5000

In [63]:
messages=X.copy()
messages.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [64]:
messages.reset_index(inplace=True)
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [65]:
import nltk
import re
from nltk.corpus import stopwords

In [66]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [52]:
from nltk.stem.porter import PorterStemmer

ps=PorterStemmer()
corpus=[]
for i in range(len(messages)):
  #print(i)
  review=re.sub('[^a-zA-z]',' ',messages['title'][i])
  review=review.lower()
  review=review.split()
  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

In [67]:
corpus[1:5]

['flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [68]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr[1:5]

[[4106, 4445, 2329, 604, 3000, 968, 4403],
 [2550, 3599, 3585, 940],
 [4899, 4564, 2901, 4238, 3076, 4464],
 [903, 3000, 4830, 1787, 2083, 4927, 3000, 3145, 1987, 2298]]

In [69]:
sent_length=20
pad_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

In [70]:
len(pad_docs)

18285

In [71]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))

model.compile( loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [72]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout_2 (Dropout)         (None, 20, 40)            0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [73]:
len(pad_docs),y.shape

(18285, (18285,))

In [74]:
import numpy as np

X_final=np.array(pad_docs)
y_final=np.array(y)

In [75]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [76]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [77]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 12s 45ms/step - loss: 0.3363 - accuracy: 0.8415 - val_loss: 0.1931 - val_accuracy: 0.9200
Epoch 2/10
192/192 [==============================] - 3s 13ms/step - loss: 0.1466 - accuracy: 0.9411 - val_loss: 0.2076 - val_accuracy: 0.9208
Epoch 3/10
192/192 [==============================] - 3s 15ms/step - loss: 0.1109 - accuracy: 0.9567 - val_loss: 0.2118 - val_accuracy: 0.9183
Epoch 4/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0852 - accuracy: 0.9704 - val_loss: 0.2221 - val_accuracy: 0.9176
Epoch 5/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0684 - accuracy: 0.9770 - val_loss: 0.2527 - val_accuracy: 0.9118
Epoch 6/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0556 - accuracy: 0.9807 - val_loss: 0.2623 - val_accuracy: 0.9122
Epoch 7/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0430 - accuracy: 0.9857 - val_loss: 0.3135 - val_accuracy: 0.9115
Ep

In [90]:
predictions = (model.predict(X_test) > 0.5).astype("int32")
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,predictions)

189/189 [==============================] - 1s 4ms/step


array([[3116,  303],
       [ 230, 2386]])

In [91]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test,predictions)

0.9116818558409279

In [80]:
embedding_vector_features=40
model2=Sequential()
model2.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model2.add(Bidirectional(LSTM(100)))
model2.add(Dropout(0.3))
model2.add(Dense(32,activation='relu'))
model2.add(Dense(1,activation='sigmoid'))

model2.compile( loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [81]:
model2.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_1 (Bidirecti  (None, 200)               112800    
 onal)                                                           
                                                                 
 dropout_4 (Dropout)         (None, 200)               0         
                                                                 
 dense_6 (Dense)             (None, 32)                6432      
                                                                 
 dense_7 (Dense)             (None, 1)                 33        
                                                                 
Total params: 319265 (1.22 MB)
Trainable params: 319265 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [82]:
model2.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 13s 46ms/step - loss: 0.2900 - accuracy: 0.8653 - val_loss: 0.1980 - val_accuracy: 0.9158
Epoch 2/10
192/192 [==============================] - 4s 19ms/step - loss: 0.1351 - accuracy: 0.9474 - val_loss: 0.2063 - val_accuracy: 0.9195
Epoch 3/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0913 - accuracy: 0.9653 - val_loss: 0.2856 - val_accuracy: 0.9142
Epoch 4/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0638 - accuracy: 0.9763 - val_loss: 0.2859 - val_accuracy: 0.9110
Epoch 5/10
192/192 [==============================] - 4s 19ms/step - loss: 0.0406 - accuracy: 0.9868 - val_loss: 0.3945 - val_accuracy: 0.9057
Epoch 6/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0311 - accuracy: 0.9892 - val_loss: 0.3993 - val_accuracy: 0.9117
Epoch 7/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0223 - accuracy: 0.9925 - val_loss: 0.4704 - val_accuracy: 0.912

In [86]:

predictions = (model2.predict(X_test) > 0.5).astype("int32")

from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,predictions)

189/189 [==============================] - 1s 4ms/step


array([[3128,  291],
       [ 249, 2367]])

In [88]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test,predictions)

0.9105219552609777